# # Import libraries

In [34]:


import requests
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import datetime


# # List of Swiss cities with coordinates   Latitude and Longitude

In [35]:
cities = [
    {"city": "Zürich", "latitude": 47.3769, "longitude": 8.5417},
    {"city": "Bern", "latitude": 46.9481, "longitude": 7.4474},
    {"city": "Genf", "latitude": 46.2044, "longitude": 6.1432},
    {"city": "Luzern", "latitude": 47.0502, "longitude": 8.3093},
    {"city": "Basel", "latitude": 47.5596, "longitude": 7.5886},
    {"city": "Lausanne", "latitude": 46.5197, "longitude": 6.6323},
    {"city": "St. Gallen", "latitude": 47.4245, "longitude": 9.3767},
    {"city": "Chur", "latitude": 46.8508, "longitude": 9.5311},
    {"city": "Lugano", "latitude": 46.0037, "longitude": 8.9511},
    {"city": "Neuenburg", "latitude": 46.9899, "longitude": 6.9293},
]

# Create a DataFrame

In [36]:
# API Base URL
BASE_URL = "https://historical-forecast-api.open-meteo.com/v1/forecast"

# Parameters for API request
params = {
    "start_date": "2024-01-01",
    "end_date": "2024-12-31",
    "daily": "temperature_2m_max",
    "timezone": "Europe/Zurich"
}

# Data collection
weather_data = []

# Fetch weather data for each city
for city in cities:
    print(f"Fetching weather data for {city['city']}...")
    params["latitude"] = city["latitude"]
    params["longitude"] = city["longitude"]

    # API Request
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        result = response.json()
        max_temperature = max(result["daily"]["temperature_2m_max"])  # Maximum temperature over the year
        weather_data.append({
            "city": city["city"],
            "latitude": city["latitude"],
            "longitude": city["longitude"],
            "max_temperature": max_temperature
        })
    else:
        print(f"Failed to fetch data for {city['city']}")

# Convert to DataFrame
weather_df = pd.DataFrame(weather_data)

# Find the city with the highest maximum temperature
hottest_city = weather_df.loc[weather_df["max_temperature"].idxmax()]

# Display the hottest city
print(f"The hottest city over the year is {hottest_city['city']} with a maximum temperature of {hottest_city['max_temperature']}°C.")

Fetching weather data for Zürich...
Fetching weather data for Bern...
Fetching weather data for Genf...
Fetching weather data for Luzern...
Fetching weather data for Basel...
Fetching weather data for Lausanne...
Fetching weather data for St. Gallen...
Fetching weather data for Chur...
Fetching weather data for Lugano...
Fetching weather data for Neuenburg...
The hottest city over the year is Genf with a maximum temperature of 35.1°C.


# Visualize geographical data

In [37]:
# Create a map centered around Switzerland
map = folium.Map(location=[46.8182, 8.2275], zoom_start=8)

# Add markers for each city
marker_cluster = MarkerCluster().add_to(map)

for index, row in weather_df.iterrows():
    color = "red" if row["max_temperature"] == hottest_city["max_temperature"] else "blue"

    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=f"{row['city']}: {row['max_temperature']}°C",
        icon=folium.Icon(color=color, icon="info-sign")
    ).add_to(marker_cluster)

# Display map directly in the notebook
from IPython.display import display
print("Displaying the map...")
display(map)

Displaying the map...
